<a href="https://colab.research.google.com/github/Gowthamipolumur/SongClassification/blob/main/audio_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install pydub

In [ ]:
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!sudo apt update
!sudo apt install ffmpeg libopus0 opus-tools

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [998 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,224 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/un

In [ ]:
import os
import subprocess
import shutil

# Function to convert any audio file to mp3 and save all mp3 files in a new directory, preserving folder structure
def convert_all_to_mp3(directory, output_directory):
    # Supported audio file extensions except mp3
    supported_extensions = ('.aac', '.opus', '.m4a', '.wav', '.flac')

    for root, dirs, files in os.walk(directory):
        for file_name in files:
            input_file = os.path.join(root, file_name)

            # Preserve the relative path in the output folder
            relative_path = os.path.relpath(root, directory)
            output_dir_with_structure = os.path.join(output_directory, relative_path)

            # Create the folder structure in the output directory if it doesn't exist
            if not os.path.exists(output_dir_with_structure):
                os.makedirs(output_dir_with_structure)

            # Destination for the converted or copied mp3 file in the new directory
            output_file = os.path.join(output_dir_with_structure, file_name.rsplit('.', 1)[0] + '.mp3')

            # If the file is already an mp3, copy it to the new directory with folder structure
            if file_name.lower().endswith('.mp3'):
                print(f"Copying {file_name} to {output_dir_with_structure}")
                shutil.copy(input_file, output_file)
                continue

            # Check if the file has a supported non-mp3 extension
            if file_name.lower().endswith(supported_extensions):
                # Temporary file for conversion
                temp_output_file = os.path.join(root, file_name.rsplit('.', 1)[0] + '_temp.mp3')

                try:
                    # Construct the ffmpeg command to convert any file to .mp3
                    command = ['ffmpeg', '-i', input_file, '-codec:a', 'libmp3lame', temp_output_file]
                    # Run the command
                    subprocess.run(command, check=True)
                    print(f"Converted {input_file} to MP3")

                    # Move the converted mp3 file to the output directory, preserving folder structure
                    shutil.move(temp_output_file, output_file)
                    print(f"Saved {output_file} to {output_dir_with_structure}")

                except subprocess.CalledProcessError as e:
                    print(f"Error converting {input_file}: {e}")

# Path to the dataset folder
dataset_folder = '/content/drive/MyDrive/DataSet Of Sounds'

# Path to the output folder for converted mp3 files, preserving folder structure
output_folder = '/content/drive/MyDrive/DataSet Of Sounds/converted_mp3'

# Convert all supported audio files in the dataset to mp3 and save to the output folder with structure
convert_all_to_mp3(dataset_folder, output_folder)


Converted /content/drive/MyDrive/DataSet Of Sounds/Baby Shark/bs9.aac to MP3
Saved /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark/bs9.mp3 to /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark
Copying bs7.mp3 to /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark
Converted /content/drive/MyDrive/DataSet Of Sounds/Baby Shark/bs8.m4a to MP3
Saved /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark/bs8.mp3 to /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark
Converted /content/drive/MyDrive/DataSet Of Sounds/Baby Shark/bs6.m4a to MP3
Saved /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark/bs6.mp3 to /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark
Copying bs5.mp3 to /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark
Converted /content/drive/MyDrive/DataSet Of Sounds/Baby Shark/bs4.m4a to MP3
Saved /content/drive/MyDrive/DataSet Of Sounds/converted_mp3/Baby Shark/bs4

In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


# 1. Data Preprocessing

# Directory of the dataset
dataset_folder = '/content/drive/MyDrive/DataSet Of Sounds/converted_mp3'


# Extract audio features using MFCC, Chroma, and Mel Spectrogram
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='scipy')  # Load audio file
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=60)  # Increase MFCC coefficients to 60
    mfccs_scaled = np.mean(mfccs.T, axis=0)

    # Extract additional features
    chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
    chroma_scaled = np.mean(chroma.T, axis=0)

    mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
    mel_scaled = np.mean(mel.T, axis=0)

    # Concatenate all features
    return np.hstack((mfccs_scaled, chroma_scaled, mel_scaled))

# Prepare dataset
def prepare_dataset(dataset_folder):
    features = []
    labels = []

    for folder in os.listdir(dataset_folder):
        folder_path = os.path.join(dataset_folder, folder)
        if os.path.isdir(folder_path):  # Check if it's a directory
            for file in os.listdir(folder_path):
                if file.endswith(".mp3"):  # Assuming files are in .mp3 format
                    file_path = os.path.join(folder_path, file)
                    # Extract features and store them with their labels
                    features.append(extract_features(file_path))
                    labels.append(folder)  # Use the folder name as the label

    return np.array(features), np.array(labels)

In [ ]:
# Load the dataset
X, y = prepare_dataset(dataset_folder)

# Encode labels to numerical values
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Feature Scaling (Normalization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# Support Vector Machine (SVM) Classifier
svm_model = SVC(kernel='linear', random_state=42)  # You can experiment with different kernels like 'rbf'
svm_model.fit(X_train, y_train)

# Evaluate SVM on validation set
y_val_pred_svm = svm_model.predict(X_val)
val_accuracy_svm = accuracy_score(y_val, y_val_pred_svm)
print(f"Validation Accuracy with SVM: {val_accuracy_svm * 100:.2f}%")

# Evaluate SVM on test set
y_test_pred_svm = svm_model.predict(X_test)
test_accuracy_svm = accuracy_score(y_test, y_test_pred_svm)
print(f"Test Accuracy with SVM: {test_accuracy_svm * 100:.2f}%")

Validation Accuracy with SVM: 57.14%
Test Accuracy with SVM: 44.44%


In [ ]:
# Random Forest Classifier (a strong classifier for structured data)
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate on validation set
y_val_pred = rf_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy with RandomForest: {val_accuracy * 100:.2f}%")

# Evaluate on test set
y_test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy with RandomForest: {test_accuracy * 100:.2f}%")

Validation Accuracy with RandomForest: 57.14%
Test Accuracy with RandomForest: 44.44%


In [1]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [2]:
!git clone https://github.com/Gowthamipolumur/SongClassification


Cloning into 'SongClassification'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [5]:
# Configure your Git username and email
!git config --global user.name "gowthamipolumur"
!git config --global user.email "gowthamipolumuri@gmail.com"


# Move the downloaded file to the cloned repository directory
#!mv audio_proj.ipynb https://github.com/Gowthamipolumur/SongClassification/


In [6]:
# Move the current notebook to the cloned repository directory
!cp "audio_proj.ipynb" "https://github.com/Gowthamipolumur/SongClassification/"

cp: cannot stat 'audio_proj.ipynb': No such file or directory
